In [1]:
from web_parser import AnswerParser, QuestionParser
import multiprocessing
from multiprocessing import Queue, Process
from selenium import webdriver
from pathlib import Path
import os
from util.repo import driver_root, database_root, drivers, git_repo_test

In [2]:
q_id = 625769474
q_list = [q_id]
config = {
    "simutaneously_scrap_and_store": False,
    "nr_answer_scapers": 2,
    "commiter_name":"test",
    "commiter_email":"test@mal.com"
}

# TODO
 - [ ] test scape then store
 - [ ] test scape and store at same time

In [3]:
class Logger:
    def __init__(self, s:str):
        self.s  = s
        
    def log(self, s):
        print("{}:{}".format(self.s,s))

In [4]:
q_task = Queue()
q_result = Queue()

In [5]:
class TestParser:
    def f(self):
        print("start")
        # return
        i = 9
        while True:
            i-=1
            print(i)
            if i<=0:
                break
                
        print("get_out")
# t = TestParser()
# t.start_parsing()

In [6]:
question_driver = webdriver.Chrome(drivers["chrome"])
question_parser = QuestionParser(
    queue_put_result=q_result,
    queue_put_task=q_task,
    driver=question_driver,
    logger=Logger("q")
)
answer_drivers = []
answer_parsers = []
for i in range(config["nr_answer_scapers"]):
    answer_drivers.append(
        webdriver.Chrome(drivers["chrome"])
    )
    answer_parsers.append(
        AnswerParser(
            queue_put_result=q_result,
            queue_get_task=q_task,
            driver=answer_drivers[i],
            logger=Logger("a_{}".format(i))
        )
    )


In [7]:
if not config["simutaneously_scrap_and_store"]:
    print("scrap, then store")
    targets = [
        # target func and args
        (question_parser.start_parsing_list, (q_list,))
    ] + [
        (answer_parser.start_parsing, [])
        for answer_parser in answer_parsers
    ]
    
    processes = [
        Process(target=target_func, args=args)
        for target_func, args in targets
    ]
    
   
    
        
        
else:
    print(1)

scrap, then store


In [8]:
for p in processes:
    p.start()
    
# for p in processes:
#     p.join()

625769474

q:parsing question with id=625769474a_0:parsing answer with id=3252328928
a_1:parsing answer with id=3249748730
parsed 625769474
fin
a_1:parsing answer with id=3252760617
a_0:parsing answer with id=3250133829
a_0:parsing answer with id=3249016418
a_1:stop
a_1:put
a_1:out
a_0:stop
a_0:put
a_0:out


In [9]:
# processes[0]._popen
# processes[0].start()
# l = processes[0]._popen

In [10]:
# type(l)

search:process queue self._wait_for_tstate_lock() 

In [9]:
from data_handler.master import Master
from data_handler.database import DataBase

In [10]:
db_path = os.path.join(database_root, "test.2.db")

In [11]:
# warning : another branch

DataBase.init_database(db_file=db_path)
# TODO: move to GitOperor.create_repo
from git import Repo
repo_ = Repo(git_repo_test)
print(repo_.head.object.hexsha)
with DataBase(db_path) as db:
    db.update_git_head(repo_.head.object.hexsha,to_commit=True)

2.6.0
2caed99f0a1d7a82f3f4ef0de9b003b1c2c76a37


In [14]:
results = []
while not q_result.empty():
# for r in q_result:
    r = q_result.get()
    results.append(r)

In [12]:
master = Master(
    db_path=db_path,
    git_repo_path=git_repo_test
)

https://stackoverflow.com/questions/71947784/what-does-webdriverwaitdriver-20-mean<br>
https://stackoverflow.com/questions/26869200/python-queue-module-get-stuck
 - [ ] test on a git repo
 - [ ] test to create git repo and make a commit

In [14]:
commiter_name =config["commiter_name"]
commiter_email = config["commiter_email"]

In [16]:
# q_result_temp = Queue()
# for r in results:
#     q_result_temp.put(r)
master.start(
    commiter_name=commiter_name,
    commiter_email=commiter_email,
    result_queue=q_result
)

tzoffset('UTC+0', -28800)
UTC
tzoffset('UTC+0', -28800)
UTC
tzoffset('UTC+0', -28800)
UTC
tzoffset('UTC+0', -28800)
UTC
tzoffset('UTC+0', -28800)
UTC
tzoffset('UTC+0', -28800)
UTC


In [16]:
print(results)

[QuestionBriefInfo(question_id=625769474, title='如何评价纪录片《当马克思遇见孔夫子》?', content=[<p data-first-child=""></p>, <figure data-size="normal"><div><img class="origin_image zh-lightbox-thumb" data-original="https://pic3.zhimg.com/v2-657d0837f091bb051f08d45ddbe07422_r.jpg" data-rawheight="1080" data-rawwidth="1917" data-size="normal" src="https://pic3.zhimg.com/v2-657d0837f091bb051f08d45ddbe07422_b.png" width="1917"/></div></figure>, <p>图源b站</p>], answerCount='72', dateCreated='2023-10-12T04:50:29.000Z', dateModified='2023-10-12T04:50:29.000Z'), AnswerBriefInfo(question_id=625769474, answer_id='3252328928', author=None, content=[<p data-first-child="" data-pid="kkerF_7K">这部片子成功达到了和春晚一样的高度</p>, <p data-pid="7ccIYuvm">不管是贴吧，b乎，抖音，微博，乃至小红书</p>, <p data-pid="bVub0cca">不管是左，中还是右</p>, <p data-pid="n_aDbji-">不管是神神还是兔兔</p>, <p data-pid="zuE2ZdNq">不管ip在哪</p>, <p data-pid="iN6Dbmjx">甚至性别都与平时不同</p>, <p data-pid="Kadlnu1W">十分罕见的一致认为这玩意是个垃圾中的垃圾</p>], upvoteCount='62', dateCreated='2023-10-16T11:44:25.000Z'

In [44]:
from dataclasses import dataclass
from typing import Union, Dict
from util.file_rendering import to_markdown, tags_to_str
import datetime

from bs4.element import Tag
# TODO: mv to util
def str2date(strdate: str) -> datetime.datetime:
    # format: "%Y-%m-%dT%H:%M:%S.%fZ"
    # example: "2023-10-13T18:14:15.000Z"
    # 
    return datetime.datetime.fromisoformat(strdate[:-1]+"+00:00")
    # return datetime.datetime.strptime(
        # strdate[:-1]+"00:00",
        # "%Y-%m-%dT%H:%M:%S.%f"
    # )
d1 = str2date("2023-10-13T18:14:15.020Z")
d2 = str2date('2023-10-12T04:50:29.001Z')


In [45]:
d1.tzinfo

datetime.timezone.utc

In [47]:
class Object:
    def __enter__(self):
        return self
    
    def __exit__(self, exc_type, exc_value, tb):
        pass
    
with Object() as o:
    print(o)
        

In [38]:
q_result_temp = Queue()
for r in results:
    q_result_temp.put(r)
    
while not q_result_temp.empty():
    print(q_result_temp.get())

In [42]:
with GitOperator(
    commiter_name=commiter_name,
    commiter_email=commiter_email,
    repo_path=git_repo_test
) as g:
    assert g is None
    print(g)

None


In [50]:
from typing import Optional
import yaml
from bs4.element import Tag

def tags_to_str(tags:list[Tag])-> str:
    return "\n".join([str(t)for t in tags])

def to_markdown(
    metadata: dict,
    content: str,
    title: Optional[str] = None
):
    metadata_yaml = yaml.dump(metadata)
    metadata_field_str = f"---\n{metadata_yaml}---\n"
    title_field_str = f"# {title}\n---\n" if title is not None else ""
    # print(f"m=={type(metadata_field_str)}\n",metadata_field_str)
    # print(f"t=={type(title_field_str)}\n",title_field_str)
    # print(f"c=={type(content)}\n",content)
    print(type(content[0]))
    return metadata_field_str + title_field_str + content
ques = results[0]
print(ques)
print(to_markdown(
    metadata={
                "type": "question",
                "question_id": ques.question_id,
                "dateCreated": ques.dateCreated,
                "answerCount": ques.answerCount,
                "title": ques.title
            },
            title=ques.title,
            content=tags_to_str(ques.content)
        ))
ans  = results[4]
print(ans)
print(to_markdown(
    metadata={
                "type": "answer",
                "answer_id": ans.answer_id,
                "author": ans.author,
                "dateCreated": ans.dateCreated,
                "upvoteCount": ans.upvoteCount,
                "commentCount": ans.commentCount
            },
            title=None,
            content=tags_to_str(ans.content)
        )
     )

QuestionBriefInfo(question_id=625769474, title='如何评价纪录片《当马克思遇见孔夫子》?', content=[<p data-first-child=""></p>, <figure data-size="normal"><div><img class="origin_image zh-lightbox-thumb" data-original="https://pic3.zhimg.com/v2-657d0837f091bb051f08d45ddbe07422_r.jpg" data-rawheight="1080" data-rawwidth="1917" data-size="normal" src="https://pic3.zhimg.com/v2-657d0837f091bb051f08d45ddbe07422_b.png" width="1917"/></div></figure>, <p>图源b站</p>], answerCount='72', dateCreated='2023-10-12T04:50:29.000Z', dateModified='2023-10-12T04:50:29.000Z')
<class 'str'>
---
answerCount: '72'
dateCreated: '2023-10-12T04:50:29.000Z'
question_id: 625769474
title: "\u5982\u4F55\u8BC4\u4EF7\u7EAA\u5F55\u7247\u300A\u5F53\u9A6C\u514B\u601D\u9047\
  \u89C1\u5B54\u592B\u5B50\u300B?"
type: question
---
# 如何评价纪录片《当马克思遇见孔夫子》?
---
<p data-first-child=""></p>
<figure data-size="normal"><div><img class="origin_image zh-lightbox-thumb" data-original="https://pic3.zhimg.com/v2-657d0837f091bb051f08d45ddbe07422_r.jpg" data-r

In [54]:
l = []
q_task_ = Queue()
for i in q_task:
    

In [55]:
l = list(q_task)

TypeError: 'Queue' object is not iterable

In [27]:
import datetime
import sqlite3
from sqlite3 import Error
from util.db import is_empty
# answer_table:
title_4_answer_table = "Answers"
title_4_question_table = "Questions"
title_4_comment_table = "Comments"
query_create_answer_table = f"""
CREATE TABLE IF NOT EXISTS {title_4_answer_table} (
    id integer PRIMARY KEY,
    authorId integer,
    questionId integer NOT NULL,
    dateCreated text
    )
"""
query_create_answer_vcs_table = f"""
CREATE TABLE IF NOT EXISTS {title_4_answer_table}VCS (
    commitId text PRIMARY KEY,
    answerId interger NOT NULL,
    dateFetched text,
    dateModified text,
    commentCount integer
    )
"""
# query_create_question_table = f"""
# CREATE TABLE IF NOT EXISTS {title_4_question_table} (
#     id integer PRIMARY KEY,
#     authorId integer,
#     dateCreated text,
#     )
# """
query_create_question_table = f"""
CREATE TABLE IF NOT EXISTS {title_4_question_table} (
    id integer PRIMARY KEY,
    authorId integer,
    dateCreated text
    )
"""
query_create_question_vcs_table = f"""
CREATE TABLE IF NOT EXISTS {title_4_question_table}VCS (
    commitId text PRIMARY KEY,
    questionId interger NOT NULL,
    dateFetched text,
    dateModified text,
    answerCount integer
    )
"""
# create a table named GitHead
# to maintain the current head
# of git repository, sha of
# which is stored;
# This table aims to prevent
# unexpected effects caused by
# manual git operations
query_create_head_table = f"""
CREATE TABLE GitHead (
    id INTEGER PRIMARY KEY CHECK (id = 0),
    sha text
);
CREATE TRIGGER git_head_no_insert
BEFORE INSERT ON GitHead
WHEN (SELECT COUNT(*) FROM GitHead) >= 1   -- limit here
BEGIN
    SELECT RAISE(FAIL, 'only one row!');
END;
"""


def init_database_tables(
    conn: sqlite3.Connection
) -> int:
    # TODO: create database file,
    # assert that file does not exist
    # if forced create: overwrite=True
    try:
        c = conn.cursor()
        print(0)
        c.execute(query_create_question_table)
        print(1)
        c.execute(query_create_answer_vcs_table)
        print(2)
        c.execute(query_create_answer_table)
        print(3)
        c.execute(query_create_question_vcs_table)
        print(4)
        c.executescript(query_create_head_table)
        print(5)
        conn.commit()
        return 0
    except Error as e:
        print(e)
        return -1


class DataBase:
    @staticmethod
    def init_database(
        db_file: str
    ):
        conn = None
        """ create a database connection to a SQLite database """
        conn = None
        try:
            conn = sqlite3.connect(db_file)
            print(sqlite3.version)
            init_database_tables(conn)
        except Error as e:
            print(e)
        finally:
            if conn:
                conn.close()

    def __init__(
        self,
        db_file: str
    ):
        self.db_file = db_file

    def __start__(self):
        self.conn = sqlite3.connect(self.db_file)

    def __exit__(self):
        self.conn.close()

    def get_git_head(self) -> str:
        c = self.conn.cursor()
        result = c.execute("""
        SELECT sha
        FROM GitHead
        """)
        l = list(result)
        return l[0][0]

    def update_git_head(self, sha: str):
        c = self.conn.cursor()
        if is_empty(c, "GitHead"):
            c.execute(f"""
            INSERT INTO GitHead
            (id, sha)
            VALUES (0,'{sha}')
            """)
        else:
            c.execute(f"""
            UPDATE GitHead
            SET sha = '{sha}'
            WHERE id==0
            """)
        self.conn.commit()

    def has_answer(self, answer_id: str) -> bool:
        c = self.conn.cursor()
        return is_empty(
            c, title_4_answer_table,
            condition=f"WHERE answerId=={answer_id}"
        )

    def has_question(self, question_id: str) -> bool:
        c = self.conn.cursor()
        return is_empty(
            c, title_4_question_table,
            condition=f"WHERE questionId=={question_id}"
        )

    def _date2str(self, d: datetime.datetime) -> str:
        # FIXME
        return str(d)

    def add_answer(
        self, answer_id: int,
        dateCreated: datetime.datetime,
        question_id: int, author_id="NULL",  # FIXME
    ):
        c = self.conn.cursor()
        dateCreated_s = self._date2str(dateCreated)
        c.executescript(f"""
        INSERT INTO {title_4_answer_table}
        (id, authorId, questionId, dateCreated)
        VALUES
        ({answer_id}, {author_id},
        {question_id}, '{dateCreated_s}')
        """)
        self.conn.commit()

    def add_question(
        self,
        dateCreated: datetime.datetime,
        question_id: int,
        author_id="NULL",  # FIXME
    ):
        c = self.conn.cursor()
        dateCreated_s = self._date2str(dateCreated)
        c.executescript(f"""
        INSERT INTO {title_4_question_table}
        (id, authorId, dateCreated)
        VALUES ({question_id}, {author_id}, '{dateCreated_s}')
        """)
        self.conn.commit()

    def add_answer_version(
        self,
        answer_id: int, commit_id: str,
        dateFetched: datetime.datetime,
        dateModified: datetime.datetime,
        commentCount: int
    ):
        #  dateFetched, dateModified,
        c = self.conn.cursor()
        dateFetched_s = self._date2str(dateFetched)
        dateModified_s = self._date2str(dateModified)
        c.executescript(f"""
        INSERT INTO {title_4_answer_table}VCS
        (commitId, answerId,
        dateFetched, dateModified,
        commentCount)
        VALUES
        ('{commit_id}',{answer_id},
        '{dateFetched_s}','{dateModified_s}',
        {commentCount})""")
        self.conn.commit()

    def add_question_version(
        self,
        question_id: int, commit_id: str,
        dateFetched: datetime.datetime,
        dateModified: datetime.datetime,
        answerCount: int
    ):
        c = self.conn.cursor()
        dateFetched_s = self._date2str(dateFetched)
        dateModified_s = self._date2str(dateModified)
        c.executescript(f"""
        INSERT INTO {title_4_question_table}VCS
        (commitId, questionId,
        dateFetched, dateModified,
        answerCount)
        VALUES
        ('{commit_id}', {question_id},
        '{dateFetched_s}', '{dateModified_s}'
        {answerCount})""")
        self.conn.commit()
